In [ ]:
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor

# 1. 기본 도구들 정의
@tool
def get_current_time(tool_input: str = "") -> str:
    """현재 시간을 반환하는 도구"""
    return "2024-11-27 15:00:00"  # 실제로는 현재 시간 계산 로직 필요

@tool
def get_current_location() -> str:
    """현재 위치를 반환하는 도구"""
    return "서울시 구로구"

@tool
def get_weather_info() -> str:
    """날씨 정보를 반환하는 도구"""
    return "맑음, 기온 20도"

# 2. WeatherAgent 클래스
class WeatherAgent:
    def __init__(self):
        # 기본 컴포넌트 초기화
        self.llm = ChatOpenAI(model="gpt-4")
        self.tools = [get_current_time, get_current_location, get_weather_info]
        
        # 프롬프트 설정
        self.prompt = ChatPromptTemplate.from_messages([ㅍ
            ("system", """날씨 정보를 제공하는 어시스턴트입니다.
            사용자의 쿼리에서 시간과 장소 정보를 추출하고,
            다음 형식으로 출력하세요:
            {
                "time": "YYYY-MM-DD HH:MM:SS",
                "location": "장소명"
            }"""),
            ("human", "{input}")
        ])
        
        # 에이전트 생성
        self.agent = self._create_agent()
    
    def _create_agent(self):
        # 에이전트 생성
        agent = create_tool_calling_agent(
            self.llm,
            self.tools,
            self.prompt
        )
        
        # 실행기 생성
        return AgentExecutor(
            agent=agent,
            tools=self.tools,
            verbose=True
        )
    
    def process_query(self, query: str) -> dict:
        """사용자 쿼리 처리"""
        try:
            # 1. 에이전트를 통해 쿼리 처리
            result = self.agent.invoke({"input": query})
            
            # 2. 결과 반환
            return {
                'schedule': result,  # 시간과 장소 정보
                'weather': get_weather_info()  # 날씨 정보
            }
        except Exception as e:
            return {
                'error': str(e)
            }

# 3. 사용 예시
def main():
    agent = WeatherAgent()
    query = "내일 날씨 어때?"
    result = agent.process_query(query)
    print(f"Query: {query}")
    print(f"Result: {result}")

if __name__ == "__main__":
    main()


테스트 케이스 #1
사용자 질문: 내일 날씨가 어떨까?
--------------------------------------------------
Schedule processing error: 1 validation error for get_current_time
tool_input
  Input should be a valid string [type=string_type, input_value={'tool_input': '내일'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
Final error handler: 1 validation error for get_current_time
tool_input
  Input should be a valid string [type=string_type, input_value={'tool_input': '내일'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
get_current_location called!
Found calendar event: 장보기

◆ 분석된 일정:
시간: 2024-11-27 15:35:19
장소: 서울시 구로구
활동: 장보기

◆ 날씨 정보 및 조언:
### 날씨 예보 (2024-11-27, 서울시 구로구, 장보기)

#### 기본 날씨:
- **온도/체감**: 최저 3도, 최고 10도
- **강수확률**: 20%
- **전반적 날씨**: 대체로 맑음
- **공기 질**: 보통

#### 상세 정보:
- **시간대별 특이사항**: 오후 시간대(15:35)에는 자외선 지수가 낮으나, 습도가 다소 높을 예정입니다.
- **활동 관련 날씨 영향**: 장보기 활동에는 대체로 쾌적한 날씨가 예상되나, 습도가 높아 다소 불쾌감을 느낄 수 있습니다.